# Intelligenza Artificiale - Lab 2

In questo laboratorio esploreremo le potenzialità del Percettrone con strati nascosti, dett anche **Multi-Layer Perceptron** (MLP). Vedremo come creare, addestrare e valutare facilmente questo tipo di modello usando una libreria Python che offre funzionalità ad alto livello per il *machine learning*: **Scikit-Learn**. Grazie a questa libreria, implementeremo un Multi-Layer Perceptron per risolvere diversi problemi di classificazione, a partire dal problema logico dello XOR fino ad arrivare a casi più realistici e complessi.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## La funzione logica XOR

Come abbiamo visto nel precedente laboratorio, un Percettrone non è in grado di risolvere il problema XOR, poiché non è linearmente separabile. Per risolverlo dobbiamo quindi aggiungere almeno uno **strato nascosto** di neuroni con **funzione di attivazione non lineare**, aumentando così le capacità espressive del modello di apprendimento.

In [ ]:
input_xor = np.array([[0, 0],
                      [0, 1],
                      [1, 0],
                      [1, 1]])
target_xor = np.array([0,
                       1,
                       1,
                       0])

In [ ]:
from sklearn.neural_network import MLPClassifier

In Scikit-Learn sia la struttura del modello che l'algoritmo di apprendimento usato per modificarne i pesi sono definiti all'interno della classe `MLPClassifier`. Quando creiamo una nuova *istanza* della classe dobbiamo quindi specificare i parametri che descrivono la struttura del modello (`hidden_layer_sizes`), quelli che specificano l'algoritmo di apprendimento usato (`solver`) e quelli che regolano l'algoritmo di apprendimento (`learning_rate_init`, `max_iter`, `random_state`).

In particolare, il parametro `random_state` (che useremo anche in altri punti di questo laboratorio) serve ad aumentare il grado di riproducibilità dei risultati che dipendono da variabili casuali, in quanto consente di ottenere gli stessi numeri random quando ripetiamo una simulazione.

Per ulteriori informazioni sulle classi e le funzioni disponibili, possiamo facilmente consultare la
[documentazione online](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) di Scikit-Learn.

In [ ]:
random_state = 42  # NB: il numero 42 è arbitrario
MLP = MLPClassifier(hidden_layer_sizes=(10),
                    solver='sgd',
                    learning_rate_init=0.1,
                    max_iter=2000,
                    random_state=random_state)

In [ ]:
MLP = MLP.fit(input_xor, target_xor.ravel())

Usando la libreria di visualizzazione `matplotlib` possiamo visualizzare l'andamento del valore della funzione di costo (loss function) durante l'apprendimento.

In [ ]:
_ = plt.plot(range(MLP.n_iter_), MLP.loss_curve_)
plt.xlabel('Epochs');
plt.ylabel('Error');
plt.ylim(0, 0.8);

La classe `MLPClassifier` ha anche un metodo `score` che permette di calcolare l'accuratezza predittiva del classificatore su un insieme di esempi tenuti da parte per la valutazione. L'accuratezza è una misura molto usata, definita semplicemente come numero di predizioni corrette sul totale.

In [ ]:
MLP.score(input_xor, target_xor) # questa funzione calcola l'accuratezza media (max = 1.0)

## Il dataset *heart disease*

Riprendiamo il dataset *heart disease* che abbiamo già utilizzato alla fine del primo laboratorio. Proviamo ora a risolvere questo problema di classificazione usando un MLP e valutiamo il suo funzionamento usando strumenti che ci permettono di capire meglio i punti di forza e di debolezza del classificatore: la curva ROC e la matrice di confusione.

In [ ]:
%%capture
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data

In [ ]:
import pandas as pd

In [ ]:
heart_disease = pd.read_csv("processed.cleveland.data",
                            names=["age", "sex", "cp", "trestbps",
                                  "chol", "fbs", "restecg", "thalach",
                                  "exang", "oldpeak", "slope", "ca", "thal", "num"],
                            na_values='?')
heart_disease = heart_disease.dropna()
heart_disease.num = heart_disease.num.apply(lambda x: 0 if x == 0 else 1)
heart_disease.head(10)

Trasformiamo gli esempi e le etichette in vettori di numeri (vettori `NumPy`). Questa volta non teniamo da parte manualmente gli esempi per il test, perché useremo una funzione automatica per partizionare il nostro dataset.

In [ ]:
hd_in  = heart_disease.loc[:, "age":"thal"].to_numpy() # le prime 13 colonne rappresentano gli input
hd_out = heart_disease.num.to_numpy(dtype=np.float64)  # l'ultima colonna rappresenta il target

### Apprendimento

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In questo caso, creiamo in automatico la divisione tra esempi di addestramento e di valutazione usando la funzione `train_test_split` inclusa nella libreria Scikit-Learn. Anche in questo caso può essere utile consultare la [documentazione online](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
(hd_in_tr, hd_in_te, hd_out_tr, hd_out_te) = train_test_split(hd_in, hd_out)

Possiamo ora procedere creando la rete neurale ed addestrandola tramite la funzione `fit`:

In [ ]:
MLP = MLPClassifier(hidden_layer_sizes=(70),  # provare ad aumentare o diminuire il numero di neuroni
                    solver='adam', learning_rate_init=0.0005, max_iter=1000, tol=0.0000001, random_state=random_state)

In [ ]:
MLP = MLP.fit(hd_in_tr, hd_out_tr)

#### Funzione di costo (loss function) e accuratezza

In [ ]:
_ = plt.plot(range(MLP.n_iter_), MLP.loss_curve_)
plt.xlabel('Epochs');
plt.ylabel('Error');
plt.ylim(0, 3);

In [ ]:
MLP.score(hd_in_tr, hd_out_tr)

In [ ]:
MLP.score(hd_in_te, hd_out_te)

#### Curva ROC

La curva ROC ([Receiver Operating Characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic/)) è un metodo grafico usato per valutare i classificatori binari. L'idea è di vedere come varia la percentuale di *true positives* e *false positives* al variare della soglia di discriminazione. Dallo stesso grafico si può anche misurare l'*Area Under the Curve* (AUC).

In [ ]:
_ = metrics.RocCurveDisplay.from_predictions(hd_out_te, MLP.predict_proba(hd_in_te)[:, 1])

#### Matrice di confusione

La matrice di confusione è un altro strumento di visualizzazione degli errori di un classificatore, applicabile anche a classificatori multi-classe. Questo metodo permette di capire in modo dettagliato la distribuzione degli errori di classificazione tra le varie classi.

In [ ]:
_ = metrics.ConfusionMatrixDisplay.from_predictions(hd_out_te, MLP.predict(hd_in_te))

## Il dataset *breast cancer*

Consideriamo ora il dataset [breast cancer](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)). Per questo problema di classificazione binaria, un MLP con uno strato nascosto e sole 10 unità riesce a risolvere il problema con una buona accuratezza.

In [ ]:
%%capture
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data

In [ ]:
names = []
names += ["ID", "label"]
cell_features = ["radius", "texture", "perimeter",
                 "area", "smoothness", "compactness", "concavity",
                 "concave_points", "symmetry", "fractal_dimension",]
for i in range(1, 4):
  names += [feature_name + f"_c{i}" for feature_name in cell_features]

In [ ]:
breast_cancer = pd.read_csv("wdbc.data",
                            names=names,
                            na_values='?')
breast_cancer = breast_cancer.dropna()
breast_cancer = breast_cancer.drop(columns=["ID"])
breast_cancer.label = breast_cancer.label.apply(lambda x: 0 if x == "B" else 1)
breast_cancer.head()

In [ ]:
bc_in = breast_cancer.loc[:, "radius_c1":"fractal_dimension_c3"].to_numpy()
bc_out = breast_cancer.label.to_numpy()

### Apprendimento

In [ ]:
(bc_in_tr, bc_in_te, bc_out_tr, bc_out_te) = train_test_split(bc_in, bc_out)

In [ ]:
MLP = MLPClassifier(hidden_layer_sizes=(50),
                    solver='adam', learning_rate_init=0.0001,
                    max_iter=1000, random_state=random_state)

In [ ]:
MLP = MLP.fit(bc_in_tr, bc_out_tr)

#### Funzione di costo (loss function) e accuratezza

In [ ]:
_ = plt.plot(range(MLP.n_iter_), MLP.loss_curve_)
plt.xlabel('Epochs');
plt.ylabel('Error');
#plt.ylim(0, 3);

In [ ]:
MLP.score(bc_in_tr, bc_out_tr)

In [ ]:
MLP.score(bc_in_te, bc_out_te)

#### Curva ROC

In [ ]:
_ = metrics.RocCurveDisplay.from_predictions(bc_out_te,
                                             MLP.predict_proba(bc_in_te)[:, 1])

#### Matrice di confusione

In [ ]:
_ = metrics.ConfusionMatrixDisplay.from_predictions(bc_out_te,
                                                    MLP.predict(bc_in_te))

## Il dataset *yeast*

Infine, consideriamo un problema di classificazione multi-classe dal dominio biologico. Il dataset [yeast](https://archive.ics.uci.edu/ml/datasets/Yeast) rappresenta diversi campioni di lieviti che possono essere classificati in 10 diverse classi.

In [ ]:
%%capture
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data

In [ ]:
yeast = pd.read_csv("yeast.data", sep='[ ]+',
                    names=["seq_name", "mcg", "gvh", "alm",
                           "mit", "erl", "pox", "vac", "nuc", "label"],
                    na_values='?', engine='python')
yeast = yeast.dropna()
yeast.head(10)

Trasformiamo i valori delle etichette in numeri interi, creando una "mappa" tra le stringhe e i numeri.

In [ ]:
yeast.label.unique()

In [ ]:
label2int = dict(zip(yeast.label.unique(), range(10)))
label2int['MIT']

In [ ]:
yeast.label = yeast.label.map(label2int)

In [ ]:
ye_in = yeast.loc[:, "mcg":"nuc"].to_numpy()
ye_out = yeast.label.to_numpy()

Come vediamo nell'istogramma sottostante, la distribuzione di esempi nelle diverse classi è molto sbilanciata, e questo avrà impatto sulle capacità predittive del nostro modello.

In [ ]:
_ = (yeast.label.value_counts()
                .sort_index()
                .plot.bar(ylabel="classe",
                          xlabel="frequenza",
                          title="Dataset yeast"))

### Apprendimento

In [ ]:
(ye_in_tr, ye_in_te, ye_out_tr, ye_out_te) = train_test_split(ye_in, ye_out)

In [ ]:
MLP = MLPClassifier(hidden_layer_sizes=(100),
                    solver='adam', learning_rate_init=0.0003,
                    max_iter=10000, random_state=random_state)

In [ ]:
MLP = MLP.fit(ye_in_tr, ye_out_tr)

#### Funzione di costo (loss function) e accuratezza

In [ ]:
_ = plt.plot(range(MLP.n_iter_), MLP.loss_curve_)
plt.xlabel('Epochs');
plt.ylabel('Error');
plt.ylim(0, 3);

In [ ]:
MLP.score(ye_in_tr, ye_out_tr)

In [ ]:
MLP.score(ye_in_te, ye_out_te)

#### Matrice di confusione

La matrice di confusione rispecchia la differenza di numerosità di esempi nelle diverse classi: la maggior parte degli errori sono infatti concentrati tra le tre classi per cui sono disponibili più esempi.

In [ ]:
_ = metrics.ConfusionMatrixDisplay.from_predictions(ye_out_te,
                                                    MLP.predict(ye_in_te))

### Convalida incrociata (cross-validation)

Questa procedura ci dà una stima piu robusta della generalizzazione del modello. Il numero e la randomizzazione delle partizioni in training/test set si può definire a priori quando si definisce l'oggetto KFold.

In [ ]:
from sklearn.model_selection import KFold
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=False)

In [ ]:
test_accuracy_folds = []
for n_test_fold, (train_index, test_index) in enumerate(kfold.split(ye_in)):
  MLP = MLPClassifier(hidden_layer_sizes=(100),
                    solver='adam', learning_rate_init=0.0003,
                    max_iter=10000, random_state=random_state)
  MLP = MLP.fit(ye_in[train_index], ye_out[train_index])
  #print(test_index)
  test_score = MLP.score(ye_in[test_index], ye_out[test_index])
  test_accuracy_folds.append(test_score)
  print(f"Accuratezza test-fold {n_test_fold}: {test_score:.3f}")

print(f"Accuratezza media del modello in cross validation: {np.mean(test_accuracy_folds):.3f}")